In [1]:
import working_day.return_working_day as wd
import discount_curve.discount_curve as dc
import csv
import pandas as pd
import datetime

def year_to_month(str_year):
        num_year = int(str_year[0:len(str_year)-1])
        return num_year * 12
    
def calc_day_count_fraction(start_date, end_date, convention):
        datetime_obj_start = datetime.datetime.strptime(start_date, '%Y/%m/%d')
        datetime_obj_end = datetime.datetime.strptime(end_date, '%Y/%m/%d')
        day_count_fraction = float((datetime_obj_end - datetime_obj_start).days / convention)
        return day_count_fraction

class swap_pricer:
    def __init__(self, valuation_date, input_file_name, convention_file_name):
        df_input = pd.read_csv(input_file_name)
        df_convention = pd.read_csv(convention_file_name, index_col=0)
        self._valuation_date = valuation_date
        self._ccy_our_pay = df_input.at[0, 'ccy_our_pay']
        self._ccy_our_rec = df_input.at[0, 'ccy_our_rec']
        self._fix_float_pay = df_input.at[0, 'fixed_or_float_pay_side']
        self._fix_float_rec = df_input.at[0, 'fixed_or_float_rec_side']
        self._term = df_input.at[0, 'term']
        self._dcc_pay = df_convention.at[self._ccy_our_pay, self._fix_float_pay]
        self._dcc_rec = df_convention.at[self._ccy_our_rec, self._fix_float_rec]
        self._roll_pay = df_convention.at[self._ccy_our_pay, 'ROLL']
        self._roll_rec = df_convention.at[self._ccy_our_rec, 'ROLL']
        self._num_pay_side = int(year_to_month(self._term) / int(self._roll_pay[0:len(self._roll_pay)-1]))
        self._num_rec_side = int(year_to_month(self._term) / int(self._roll_rec[0:len(self._roll_rec)-1]))
        self._names_float =  ['start_date', 'end_date', 'payment_date', 'dcf', 'DF', 'foward rate', 'PV']
        self._names_fix = ['start_date', 'end_date', 'payment_date', 'dcf', 'DF', 'fixed rate', 'PV']

    def _create_df_pay(self):
        index_pay = range(self._num_pay_side+1)
        if (self._fix_float_pay=='FLOAT'):
            df_pay_side = pd.DataFrame(index=index_pay, columns=self._names_float)
        elif(self._fix_float_pay=='FIXED'):
            df_pay_side = pd.DataFrame(index=index_pay, columns=self._names_fix)
        return df_pay_side
    
    def _create_df_rec(self):
        index_rec = range(self._num_rec_side+1)
        if (self._fix_float_rec=='FLOAT'):
            df_rec_side = pd.DataFrame(index=index_rec, columns=self._names_float)
        elif(self._fix_float_rec=='FIXED'):
            df_rec_side = pd.DataFrame(index=index_rec, columns=self._names_fix)
        return df_rec_side        
    
    def add_cash_flow_pay(self, IR_data_name):
        df_pay_side = self._create_df_pay()
        end_date_list = wd.create_end_date(self._valuation_date, self._roll_pay, 'following', self._num_pay_side, 2)
        start_date_list = wd.create_start_date(self._valuation_date, self._roll_pay, 'following', self._num_pay_side, 2)
        DF_obj = dc.discount_factor(IR_data_name, self._ccy_our_pay, self._valuation_date)
        DF_list = DF_obj.get_DF_list()
        df_DF = pd.DataFrame(DF_list, columns=['num', 'date', 'DF'])
        df_DF_date_index = df_DF.set_index('date')
        for i in range(self._num_pay_side+1):
            df_pay_side['start_date'][i] = start_date_list[i]
            df_pay_side['end_date'][i] = end_date_list[i]
            df_pay_side['payment_date'][i] = end_date_list[i]
            df_pay_side['dcf'][i] = calc_day_count_fraction(start_date_list[i], end_date_list[i], int(self._dcc_pay[-3:]))
            df_pay_side['DF'][i] = df_DF_date_index.at[end_date_list[i], 'DF']
            df_pay_side['foward rate'][i] = (df_DF_date_index.at[start_date_list[i], 'DF'] / df_DF_date_index.at[end_date_list[i], 'DF'] -1) /  df_pay_side['dcf'][i]
            df_pay_side['PV'][i] =  df_pay_side['dcf'][i] *  df_pay_side['foward rate'][i] * df_pay_side['DF'][i]
        return df_pay_side
        
    def get_ccy_our_pay(self):
        return self._ccy_our_pay
    
    def get_ccy_our_rec(self):
        return self._ccy_our_rec
    
    def get_fix_float_pay(self):
        return self._fix_float_pay
    
    def get_fix_float_rec(self):
        return self._fix_float_rec
    
    def get_term(self):
        return self._term
    
    def get_dcc_pay(self):
        return self._dcc_pay
    
    def get_dcc_rec(self):
        return self._dcc_rec
    
    def get_num_pay_side(self):
        return self._num_pay_side 
    
    def get_num_rec_side(self):
        return self._num_rec_side 
    
    def get_df_pay_side(self):
        df_pay = self._create_df_pay()
        return df_pay
    
    def get_df_rec_side(self):
        df_rec = self._create_df_rec()
        return df_rec

In [2]:
price_obj = swap_pricer('2018/01/18', 'swap_pricing_input.csv', 'convention.csv')
price_obj.get_num_rec_side()
price_obj.get_df_pay_side()
test_df = price_obj.get_df_rec_side()

In [3]:
price_obj.add_cash_flow_pay('IR_data.csv')

,start_date,end_date,payment_date,dcf,DF,foward rate,PV
0,2018/01/18,2018/01/20,2018/01/20,0.00555556,0.999921,0.0142967,7.94197e-05
1,2018/01/20,2018/07/18,2018/07/18,0.497222,0.990573,0.0189793,0.00934799
2,2018/07/18,2019/01/18,2019/01/18,0.511111,0.978687,0.0237607,0.0118855
3,2019/01/18,2019/07/18,2019/07/18,0.502778,0.937335,0.0877466,0.0413524
4,2019/07/18,2020/01/20,2020/01/20,0.516667,0.923553,0.0288821,0.0137816
5,2020/01/20,2020/07/21,2020/07/21,0.508333,0.911064,0.026966,0.0124886
6,2020/07/21,2021/01/18,2021/01/18,0.502778,0.898445,0.0279354,0.0126189
7,2021/01/18,2021/07/20,2021/07/20,0.508333,0.886415,0.0266983,0.0120301
8,2021/07/20,2022/01/18,2022/01/18,0.505556,0.874379,0.0272277,0.0120359
9,2022/01/18,2022/07/19,2022/07/19,0.505556,0.862732,0.0267034,0.0116469


In [34]:
'ACT/365'[-3:]

'365'

In [10]:
import pandas as pd

In [6]:
df_convention = pd.read_csv('convention.csv', index_col=0)
df_convention

,FIXED,FLOAT,ROLL
CCY,,,
EUR,ACT/365,ACT/360,6M
GBP,ACT/365,ACT/360,6M
JPY,ACT/365,ACT/360,6M
USD,ACT/365,ACT/360,3M


In [8]:
df_convention.at['JPY', 'FLOAT']

'ACT/360'

In [9]:
df_input = pd.read_csv('swap_pricing_input.csv')
df_input.at[0, 'ccy_our_pay']

'JPY'

In [95]:
year_to_month('10Y')

120

In [17]:
names_float = ['start_date', 'end_date', 'dcc', 'payment_date', 'DF', 'foward rate', 'PV']
index_float = range(21)
test_df = pd.DataFrame(index=index_float, columns=names_float)
test_df

,start_date,end_date,dcc,payment_date,DF,foward rate,PV
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# backup 2/18

In [54]:
'''    def _input_csv_read(self, input_file_name):
        with open(input_file_name, 'r') as csvfile:
            reader_obj = csv.reader(csvfile)
            input_list = []
            for row in reader_obj:
                input_list.append(row)
        key = input_list[0]
        value = input_list[1]
        input_dict = dict(zip(key, value))
        return input_dict
'''

"    def _input_csv_read(self, input_file_name):\n        with open(input_file_name, 'r') as csvfile:\n            reader_obj = csv.reader(csvfile)\n            input_list = []\n            for row in reader_obj:\n                input_list.append(row)\n        key = input_list[0]\n        value = input_list[1]\n        input_dict = dict(zip(key, value))\n        return input_dict\n"